In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from scombi_do.app import scombi
import sys
sys.path.append('..')
from helpers import item_to_img_overlay, my_read_method
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType, extensions
from shapely.geometry import shape
from ipyleaflet import Map

### Stage the STAC catalogs

In [59]:
input_references = ['https://terradue-stac.gitlab.io/landsat-8-l1/098/85/LC80980852020009/LC80980852020009.json']

In [60]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(catalog.get_self_href())

2020-12-05T17:45:24 DEBUG    Starting new HTTPS connection (1): terradue-stac.gitlab.io:443
2020-12-05T17:45:24 DEBUG    https://terradue-stac.gitlab.io:443 "GET /landsat-8-l1/098/85/LC80980852020009/LC80980852020009.json HTTP/1.1" 200 10438


* <Catalog id=LC80980852020009>
  * <Item id=LC80980852020009>


In [61]:
catalogs

['/workspace/scombi-do/demo/notebook/LC80980852020009/catalog.json']

### Scombi-do

#### Simple RGB combination

The reflectances are encoded with a scaling factor of 10000. 

The S-expressions transform that value from [0,10000] to [0,1] using the numpy `interp` function

S-expressions (or symbolic expressions, abbreviated as sexprs) are a notation for nested list (tree-structured) data, invented for and popularized by the programming language Lisp

`v1`, `v2` and `v3` are the numpy arrays read from the input product band

In [62]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [63]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [64]:
channel_inputs = [catalogs[0], 
                  catalogs[0],
                  catalogs[0]]
channel_inputs

['/workspace/scombi-do/demo/notebook/LC80980852020009/catalog.json',
 '/workspace/scombi-do/demo/notebook/LC80980852020009/catalog.json',
 '/workspace/scombi-do/demo/notebook/LC80980852020009/catalog.json']

In [65]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [66]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [67]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [68]:
result = scombi(**params)

2020-12-05T17:45:33 INFO     <Item id=LC80980852020009>
2020-12-05T17:45:33 INFO     <Item id=LC80980852020009>
2020-12-05T17:45:33 INFO     <Item id=LC80980852020009>
2020-12-05T17:45:56 INFO     Rescaling and COG for input assets
2020-12-05T17:45:56 INFO     Getting band red from /vsicurl/https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/098/085/LC08_L1TP_098085_20200109_20200109_01_RT/LC08_L1TP_098085_20200109_20200109_01_RT_B4.TIF
2020-12-05T17:46:19 INFO     Getting band green from /vsicurl/https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/098/085/LC08_L1TP_098085_20200109_20200109_01_RT/LC08_L1TP_098085_20200109_20200109_01_RT_B3.TIF
2020-12-05T17:46:40 INFO     Getting band blue from /vsicurl/https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/098/085/LC08_L1TP_098085_20200109_20200109_01_RT/LC08_L1TP_098085_20200109_20200109_01_RT_B2.TIF
2020-12-05T17:47:00 INFO     Build VRT
2020-12-05T17:47:00 INFO     30.0
2020-12-05T17:47:00 INFO     Pimp me
2020-12-05T17:47:00 INFO 

In [69]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)

m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

#### Normalized difference

In [ ]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[0]]
channel_inputs

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

### Multitemporal RGB composite

In [ ]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[1],
                  catalogs[2]]
channel_inputs

In [ ]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45' 

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [27]:
result = scombi(**params)

2020-12-04T17:04:45 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2020/2/S2B_53HPA_20200209_0_L2A/B02.tif
2020-12-04T17:05:46 INFO     Build VRT
2020-12-04T17:05:46 INFO     10.0
2020-12-04T17:05:46 INFO     Pimp me
2020-12-04T17:05:46 INFO     3
2020-12-04T17:05:47 INFO     Applying color operations: Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45
2020-12-04T17:05:54 INFO     Adding band 1 of 3
2020-12-04T17:05:54 INFO     Adding band 2 of 3
2020-12-04T17:05:54 INFO     Adding band 3 of 3
2020-12-04T17:05:59 INFO     STAC
2020-12-04T17:05:59 INFO     Done!


In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m